В этом блокноте мы сконструируем графовую нейронную сеть, используя PyTorch Geometric (PyG) и применим модель к двум наборам данных из Open Graph Benchmark (OGB). Эти два набора данных используются для сравнения производительности моделей на двух различных задачах машинного обучения на графах. Одна из них - предсказание свойств вершины, относящаяся к отдельным вершинам. Другая - предсказание свойств графа, относящаяся к графам и их (частям) суб-графам.

Для начала мы ознакомимся с тем, каким образом PyTorch Geometric хранит графы в тензорах PyTorch.

Затем - загрузим один из наборов данных Open Graph Benchmark (OGB) с помощью пакета `ogb`. OGB - это коллекция реалистичных больших и разнообразных эталонных наборов данных для машинного обучения на графах. Пакет `ogb` предоставляет не только загрузчик данных, но и средства оценки моделей.

Наконец, мы построим нашу собственную графовую нейронную сеть с помощью PyTorch Geometric. Затем - применим и оценим качество работы моделей в задаче предсказания свойств вершин и графов.

In [121]:
import torch
print(torch.__version__)

1.12.1


# Установка

In [ ]:
#!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
# #!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
# !pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
# !pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
# !pip install -q torch-geometric
# !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
# !pip install ogb

# 1 PyTorch Geometric (наборы и формат данных)


PyTorch Geometric содержит два класса для хранения и трансформации графов в формат тензоров. Один, `torch_geometric.datasets`, содержит набор общеизвестных наборов графовых данных. Другой - это `torch_geometric.data`, позволяющий работать с графами в тензорах PyTorch.

В данной секции мы разберёмся с тем, как использовать `torch_geometric.datasets` и `torch_geometric.data`.

## Наборы данных PyG

Модуль `torch_geometric.datasets` содержит множество общеизвестных наборов графовых данных. Здесь мы ознакомимся с функциональностью на примере данных об энзимах.

In [122]:
from torch_geometric.datasets import TUDataset

root = './enzymes'
name = 'ENZYMES'

# Набор данных ENZYMES
pyg_dataset= TUDataset('./enzymes', 'ENZYMES')

# Как видим, в данном наборе 600 графов
print(pyg_dataset)

ENZYMES(600)


In [123]:
def get_num_classes(pyg_dataset):
    """Функция принимает объект набора данных PyG и возвращающую количество классов для этого набора данных."""
    try:
        num_classes = pyg_dataset.num_classes
    except:
        num_classes = 0

    return num_classes


def get_num_features(pyg_dataset):
    """Функция принимает объект набора данных PyG и возвращающую количество свойств для этого набора данных."""
    try:
        num_features = pyg_dataset.num_features
    except:
        num_features = 0

    return num_features

# Чему равны количество классов и количество свойств набора данных в ENZYMES?

num_classes = get_num_classes(pyg_dataset)
num_features = get_num_features(pyg_dataset)
print("{} dataset has {} classes".format(name, num_classes))
print("{} dataset has {} features".format(name, num_features))

ENZYMES dataset has 6 classes
ENZYMES dataset has 3 features


## Данные в PyG

Каждый набор данных в PyG обычно содержит список объектов `torch_geometric.data.Data`. Каждый объект `torch_geometric.data.Data` обычно представляет собой граф. Вы можете легко получить объект `Data`, проиндексировав набор данных.

Для более подробной информации о том, что может содержать объект `Data`, пожалуйста обратитесь к [документации](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data).

In [124]:
def get_graph_class(pyg_dataset, idx):
    """Функция принимает объект набора данных PyG и индекс графа в нём, и возвращающую класс/метку графа (целое число)."""
    try:
        label = pyg_dataset[idx].y.item()
    except:
        label = -1

    return label

# Здесь pyg_dataset - набор для классификации графов
graph_0 = pyg_dataset[0]
print(graph_0)
idx = 100
label = get_graph_class(pyg_dataset, idx)

# Какова метка (label) графа (с индексом 100 в наборе данных ENZYMES)?
print('Graph with index {} has label {}'.format(idx, label))

Data(edge_index=[2, 168], x=[37, 3], y=[1])
Graph with index 100 has label 4


In [125]:
def get_graph_num_edges(pyg_dataset, idx):
    """Функцию, принимающую объект набора данных PyG и индекс графа в нём, и возвращающую количество рёбер графа (целое число).

    Не должны считаться рёбра дважды для неориентированных графов. Например, в неориентированном графе G, 
    если две вершины v и u соединены ребром, его должно посчитать лишь раз.
    """
    num_edges = 0

    try:
        num_edges = pyg_dataset[idx].num_edges // 2
    except:
        num_edges = 0

    return num_edges

# Чему равно количество рёбер графа (индекс 200 в наборе данных ENZYMES)?
idx = 200
num_edges = get_graph_num_edges(pyg_dataset, idx)
print('Graph with index {} has {} edges'.format(idx, num_edges))

Graph with index 200 has 53 edges


# 2 Open Graph Benchmark (OGB)

Open Graph Benchmark (OGB) - это коллекция реалистичных, масштабных и разнообразных канонических наборов данных для машинного обучения на графах. Наборы данных из неё автоматически загружаются, обрабатываются и разбиваются с помощью OGB Data Loader. Качество модели также может быть оценено с помощью OGB Evaluator унифицированным образом.

## Наборы и формат данных

OGB также поддерживает наборы и формат данных PyG. Ознакомимся с содержимым набора данных `ogbn-arxiv`.

In [126]:
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset

dataset_name = 'ogbn-arxiv'
# Загрузим данные и транфсормируем в разреженный тензор
dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())
print('The {} dataset has {} graph'.format(dataset_name, len(dataset)))

# Добудем граф
data = dataset[0]
print(data)

The ogbn-arxiv dataset has 1 graph
Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1166243])


In [127]:
def graph_num_features(data):
    """Функция принимает объект набора данных PyG и индекс графа в нём.
    
    Возвращающую количество свойств графа (целое число).
    """
    try:
        num_features = data.num_features
    except:
        num_features = 0

    return num_features

# Каково количество свойств в графе ogbn-arxiv?

num_features = graph_num_features(data)
print('The graph has {} features'.format(num_features))

The graph has 128 features


# 3 GNN: Предскажем свойства вершин

В данной секции мы построим нашу первую графовую нейронную сеть, используя PyTorch Geometric и применим её к задаче предсказания свойств вершин (node classification).

Мы построим эту графовую нейронную сеть, используя оператор свёртки (GCN) ([Kipf et al. (2017)](https://arxiv.org/pdf/1609.02907.pdf)).

Вы должны непосредственно использовать встроенный в PyG слой `GCNConv`. 

In [128]:
import torch
import torch.nn.functional as F
print(torch.__version__)

# The PyG built-in GCNConv
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

1.12.1


## Загрузим и предобработаем набор данных

In [129]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())
data = dataset[0]

# Сделаем матрицу связности симметричной
data.adj_t = data.adj_t.to_symmetric()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Если вы используете GPU, device должен быть cuda
print('Device: {}'.format(device))

data = data.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

Device: cpu


In [130]:
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

## GCN - графовая свёрточная сеть

Теперь мы реализуем нашу модель GCN!

Пожалуйста, реализуйте функцию прямого прохода согласно схеме на рисунке.


![test](https://drive.google.com/uc?id=128AuYAXNXGg7PIhJJ7e420DoPWKb-RtL)

In [131]:
import torch_geometric

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):

        super(GCN, self).__init__()

        # Список слоёв GCNConv
        self.convs = torch.nn.ModuleList(
            [GCNConv(input_dim, hidden_dim)] + \
            [GCNConv(hidden_dim, hidden_dim) for i in range(num_layers - 2)] +\
            [GCNConv(hidden_dim, output_dim)]
        )
        
        # Список слоёв 1D batch normalization
        self.bns = torch.nn.ModuleList([torch.nn.BatchNorm1d(hidden_dim) for i in range(num_layers - 1)])
        
        # Слой softmax
        self.softmax = torch.nn.LogSoftmax(dim = -1)

        # Вероятность обнуления элемента
        self.dropout = dropout

        # Возвращаем эмбеддинги в обход слоя классификатора
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()


    def forward(self, x, adj_t):
        """Функция принимает на вход тензор x, edge_index тензор adj_t и возвращает выходной тензор как показано на рисунке"""

        for i in range(len(self.bns)):
            x = self.convs[i](x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p = self.dropout, training = self.training)
        x = self.convs[-1](x, adj_t)
        
        if not self.return_embeds: 
            x = self.softmax(x)
            
        out = x

        return out

In [132]:
def train(model, data, train_idx, optimizer, loss_fn):
    model.train()
    loss = 0

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)
    out, y = out[train_idx], torch.flatten(data.y[train_idx])
    loss = loss_fn(out, y)
    loss.backward()
    optimizer.step()

    return loss.item()

In [133]:
# Протестируем функцию

@torch.no_grad()
def test(model, data, split_idx, evaluator):
    """Функция тестирования модели,использующая заданные split_idx и evaluator"""
    model.eval()

    out = model(data.x, data.adj_t)

    #########################################
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [134]:
args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 10,
}
args

{'device': 'cpu',
 'num_layers': 3,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.01,
 'epochs': 10}

In [136]:
model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbn-arxiv')

In [137]:
import copy

# Сбросим параметры к изначальным псевдослучайным значениям
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
    loss = train(model, data, train_idx, optimizer, loss_fn)
    result = test(model, data, split_idx, evaluator)
    train_acc, valid_acc, test_acc = result
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Epoch: 01, Loss: 3.9159, Train: 26.04%, Valid: 29.31% Test: 26.25%
Epoch: 02, Loss: 2.3869, Train: 25.65%, Valid: 23.24% Test: 28.19%
Epoch: 03, Loss: 1.9189, Train: 23.17%, Valid: 18.09% Test: 16.00%
Epoch: 04, Loss: 1.8019, Train: 33.75%, Valid: 24.54% Test: 21.75%
Epoch: 05, Loss: 1.6685, Train: 38.26%, Valid: 30.12% Test: 28.60%
Epoch: 06, Loss: 1.5693, Train: 35.57%, Valid: 28.31% Test: 29.65%
Epoch: 07, Loss: 1.5066, Train: 36.93%, Valid: 32.66% Test: 36.22%
Epoch: 08, Loss: 1.4510, Train: 37.73%, Valid: 33.82% Test: 38.61%
Epoch: 09, Loss: 1.4106, Train: 37.53%, Valid: 32.08% Test: 36.67%
Epoch: 10, Loss: 1.3805, Train: 37.36%, Valid: 31.14% Test: 36.01%


In [138]:
best_result = test(best_model, data, split_idx, evaluator)
train_acc, valid_acc, test_acc = best_result
print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Best model: Train: 37.73%, Valid: 33.82% Test: 38.61%


# 4 GNN: Предскажем свойства графа

В этой секции мы создадим графовую нейронную сеть для предсказания свойств граф (graph classification)


## Загрузим и предобработаем набор данных

Наборы данных ogbg-molhiv и ogbg-molpcba — это два набора данных для прогнозирования молекулярных свойств разного размера.

Они взяты из MoleculeNet и являются одними из крупнейших наборов данных MoleculeNet. 

Все молекулы предварительно обрабатываются с помощью RDKit. 

Каждый граф представляет собой молекулу, узлами которой являются атомы, а ребрами — химические связи. Характеристики входного узла являются 9-мерными и содержат атомный номер и хиральность, а также другие дополнительные характеристики атома, такие как формальный заряд и то, находится ли атом в кольце или нет.

**Задача прогнозирования:** Задача состоит в том, чтобы как можно точнее предсказать целевые молекулярные свойства, при этом молекулярные свойства представлены в виде бинарных меток, например, ингибирует ли молекула репликацию вируса ВИЧ или нет. 

In [139]:
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.data import DataLoader
from tqdm.notebook import tqdm

# Загрузим набор данных 
dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

split_idx = dataset.get_idx_split()

# Проверим тип задачи
print('Task type: {}'.format(dataset.task_type))

Device: cpu
Task type: binary classification


In [140]:
# Загрузим наборы данных в загрузчик
# Мы проведём тренировку классификации графов на партии (batch) из 32 графов
# Перемешаем порядок графов для обучающей выборки
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False, num_workers=0)

/opt/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [141]:
# Пожалуйста, не меняйте параметры args
args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 5,
}
args

{'device': 'cpu',
 'num_layers': 5,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.001,
 'epochs': 5}

## Graph Prediction Model

Теперь мы реализуем нашу модель, предсказывающую свойства графа!

Мы задействуем уже реализованную GCN для производства векторных представлений вершин `node_embeddings` и воспользуемся Global Pooling свойств вершин для предсказания свойств графа.

In [142]:
from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_add_pool, global_mean_pool

### GCN для предсказания свойств графа
class GCN_Graph(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers, dropout):
        super(GCN_Graph, self).__init__()

        # Загрузим кодировщики для атомов в графах молекул
        self.node_encoder = AtomEncoder(hidden_dim)

        # Модель эмбеддингов вершин
        # Обратите внимание: input_dim и output_dim установлены как hidden_dim
        self.gnn_node = GCN(hidden_dim, hidden_dim,
            hidden_dim, num_layers, dropout, return_embeds=True)

        self.pool = None

        self.pool = global_mean_pool

        # Выходной слой
        self.linear = torch.nn.Linear(hidden_dim, output_dim)


    def reset_parameters(self):
        self.gnn_node.reset_parameters()
        self.linear.reset_parameters()


    def forward(self, batched_data):
        """Функция принимает тензор batched_data и возвращает <<a batched output tensor>> для каждого графа"""
        x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch
        embed = self.node_encoder(x)

        embed = self.gnn_node(embed, edge_index)
        embed = self.pool(embed, batch)
        out = self.linear(embed)

        return out

In [143]:
def train(model, device, data_loader, optimizer, loss_fn):
    model.train()
    loss = 0

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
              pass
        else:
            ## игнорирурем nan метки классов (неразмеченные) для вычисления функции потерь (training loss)
            is_labeled = batch.y == batch.y

            optimizer.zero_grad()
            out = model(batch)
            loss = loss_fn(out[is_labeled], batch.y[is_labeled].float())

            loss.backward()
            optimizer.step()

    return loss.item()

In [144]:
def eval(model, device, loader, evaluator):
    model.eval()
    y_true = []
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            y_true.append(batch.y.view(pred.shape).detach().cpu())
            y_pred.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()

    input_dict = {"y_true": y_true, "y_pred": y_pred}

    return evaluator.eval(input_dict)

In [145]:
model = GCN_Graph(args['hidden_dim'],
            dataset.num_tasks, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbg-molhiv')

In [146]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
    print('Training...')
    loss = train(model, device, train_loader, optimizer, loss_fn)

    print('Evaluating...')
    train_result = eval(model, device, train_loader, evaluator)
    val_result = eval(model, device, valid_loader, evaluator)
    test_result = eval(model, device, test_loader, evaluator)

    train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Training...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

...

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Epoch: 05, Loss: 0.9590, Train: 78.67%, Valid: 76.63% Test: 72.77%


In [ ]:
train_acc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
valid_acc = eval(best_model, device, valid_loader, evaluator)[dataset.eval_metric]
test_acc = eval(best_model, device, test_loader, evaluator)[dataset.eval_metric]

print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

### Спасибо за внимание!